In [1]:
import os
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
import cv2
import numpy as np
import argparse
import skimage
import scipy.io as sio
import import_ipynb
import glob
import math
import pickle
import matplotlib.pyplot as plt
from mywarper import warp
from skimage import io, transform

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.autograd import Variable

parser = argparse.ArgumentParser(description='stat231_project1')
parser.add_argument('--epochs', type=int, default=300)
parser.add_argument('--batch_size', type=int, default=100)
parser.add_argument('--seed', type=int, default=12345)
parser.add_argument('--device', type=int, default=0)
parser.add_argument('--image_dir', type=str, default='./images/')
parser.add_argument('--landmark_dir', type=str, default='./landmarks/')
parser.add_argument('--male_img_dir', type=str, default='./male_images/')
parser.add_argument('--female_img_dir', type=str, default='./female_images/')
parser.add_argument('--male_landmark', type=str, default='./male_landmarks/')
parser.add_argument('--female_landmark', type=str, default='./female_landmarks/')
parser.add_argument('--path', type=str, default='./results/model/')
parser.add_argument('--log', type=str, default='./results/log/')
parser.add_argument('--appear_lr', type=float, default=7e-4)
parser.add_argument('--landmark_lr', type=float, default=1e-4)

# Read Dataset
class data_reader(object):
    def __init__(self, root_dir, file_str_len, origin_name, file_format):
        self.root_dir = root_dir
        self.file_str_len = file_str_len
        self.origin_name = origin_name
        self.file_format = file_format

    def read(self, split, read_type):
        files_len = len([name for name in os.listdir(self.root_dir) 
                        if os.path.isfile(os.path.join(self.root_dir, name))])
        counter = 0
        idx = counter
        dataset = []
        train_dataset = []
        test_dataset = []
        while counter < files_len:
            name = self.origin_name + str(idx)
            if len(name) > self.file_str_len:
                name = name[len(name)-self.file_str_len:]
            try:
                if read_type == 'image':
                    data = io.imread(self.root_dir + name + self.file_format)
                elif read_type == 'landmark':
                    mat_data = sio.loadmat(self.root_dir + name + self.file_format)

                    data = mat_data['lms']
                dataset.append(data)
                counter += 1
            except FileNotFoundError:
                pass
            idx += 1
        train_dataset = dataset[:split]
        test_dataset = dataset[split:]
        return train_dataset, test_dataset

# Construct Dataset
class ImgToTensor(object):
    def __call__(self, sample):
        sample = sample.transpose((2, 0, 1))
        return torch.tensor(sample, dtype=torch.float32)/255

class LandmarkToTensor(object):
    def __call__(self, sample):
        return torch.tensor(sample, dtype=torch.float32)/128

class dataset_constructor(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample_data = self.dataset[idx]
        if self.transform:
            sample_data = self.transform(sample_data)
        return sample_data

'''
CONV layer: computes output of neurons connected to local regions in input,
each layer computing a dot product between its weights and the small region
to which it is connected in the input volume. This may result in volume such
as [32x32x12] if we decided to use 12 filters.
ReLU layer: applies an element-wise activation function, such as the max(0,x)
thresholding at zero. This leaves the size of the volume unchanged ([32x32x12]).
POOL layer: performs a downsampling operation along the spatial dimensions (width, height),
resulting in volume such as [16x16x12].
FC (fully-connected) layer: computes the class scores, resulting in volume of size [1x1x10],
in which each of the 10 numbers corresponds to a class score, e.g. among the 10 categories of CIFAR-10.
As with ordinary Neural Networks, each neuron in this layer is connected to all numbers in previous volume.

CONV/FC: perform transformations based on activations of input volume and weights and biases of neurons.
Parameters are trained with gradient descent so that class scores are consistent with labels in training set.
ReLU/POOL: implement a fixed function
CONV/FC/POOL: may have hyperparameters (ReLU does not)
'''

APP_LATENT_Z = []
GEO_LATENT_Z = []

# Convolutional architecture, reconstructs and generates 2-d face images
class appearance_autoencoder(nn.Module):
    def __init__(self):
        super(appearance_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            # Applies a 2D convolution over input signal composed of several input planes
            # params: (in_channels, out_channels, kernel_size, stride=1, padding=0, ...)
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
        )

        self.fc1 = nn.Sequential(
            nn.Linear(128 * 8 * 8, 50),
            nn.LeakyReLU(),
        )

        self.decoder = nn.Sequential(
            # De-Conv in PyTorch: ConvTranspose2d
            nn.ConvTranspose2d(50, 128, kernel_size=8, stride=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16 ,3, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        latent_z = self.fc1(x.view(-1, 128 * 8 * 8))
        APP_LATENT_Z.append(latent_z)
        x_recon = self.decoder(latent_z.view(-1, 50, 1, 1))
        return x_recon

# Fully-connected architecture, reconstructs and generates landmarks
class landmark_autoencoder(nn.Module):
    def __init__(self):
        super(landmark_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            # Applies a linear transformation to the incoming data: y = x*A' + b
            nn.Linear(68 * 2, 100),
            # Applies element-wise LeakyReLU(x) = max(0,x) + negative_slope * min(0,x)
            nn.LeakyReLU(),
            nn.Linear(100, 10),
            nn.LeakyReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 100),
            nn.LeakyReLU(),
            nn.Linear(100, 68 * 2),
            # Applies element-wise Sigmoid(x) = 1 / (1 + exp(−x))
            nn.Sigmoid(),
        )

    def forward(self, x):
        latent_z = self.encoder(x)
        GEO_LATENT_Z.append(latent_z)
        x_recon = self.decoder(latent_z)
        return x_recon

class autoencoder(object):
    def __init__(self, appear_lr, landmark_lr, use_cuda):
        self.appear_model = appearance_autoencoder()
        self.landmark_model = landmark_autoencoder()
        self.use_cuda = use_cuda
        if use_cuda:
            self.appear_model.cuda()
            self.landmark_model.cuda()
        self.criterion = nn.MSELoss() # MSELoss loss = (x_n − y_n)^2
        self.appear_optim = optim.Adam(self.appear_model.parameters(), lr=appear_lr)
        self.landmark_optim = optim.Adam(self.landmark_model.parameters(), lr=landmark_lr)
        
    def train_appear_model(self, epochs, trainloader):
        self.appear_model.train()
        recon = []
        final_loss = 0
        for epoch in range(0, epochs):
            final_epoch = epoch == epochs - 1
            training_loss = 0
            for batch in trainloader:
                self.appear_optim.zero_grad()
                batch_recon = self.appear_model(batch)
                if final_epoch:
                    recon.append(batch_recon)
                loss = self.criterion(batch_recon, batch)
                loss.backward()
                # Updates parameters, can be called once gradients are computed with backward()
                self.appear_optim.step()
                training_loss += loss.item()
            if final_epoch: 
                final_loss = training_loss
            print('Training Appearance Epoch: {}, Loss: {:.6f}'.format(epoch, training_loss / len(trainloader)))
        return recon, final_loss, self

    def train_landmark_model(self, epochs, trainloader):
        self.landmark_model.train()
        recon = []
        final_loss = 0
        for epoch in range(0, epochs):
            final_epoch = epoch == epochs - 1
            training_loss = 0
            for batch in trainloader:
                self.landmark_optim.zero_grad()
                batch_recon = self.landmark_model(batch)
                if final_epoch:
                    recon.append(batch_recon)
                loss = self.criterion(batch_recon, batch)
                loss.backward()
                self.landmark_optim.step()
                training_loss += loss.item()
            if final_epoch:
                final_loss = training_loss
            print('Training Landmark Epoch: {}, Loss: {:.6f}'.format(epoch, training_loss / len(trainloader)))
        return recon, final_loss
        
    def test_appear_model(self, testloader):
        self.appear_model.eval()
        recon = []
        training_loss = 0
        for batch in testloader:
            batch_recon = self.appear_model(batch)
            recon.append(batch_recon)
            loss = self.criterion(batch_recon, batch)
            training_loss += loss.item()
        return recon, training_loss
    
    def test_landmark_model(self, testloader):
        self.landmark_model.eval()
        recon = []
        training_loss = 0
        for batch in testloader:
            batch_recon = self.landmark_model(batch)
            recon.append(batch_recon)
            loss = self.criterion(batch_recon, batch)
            training_loss += loss.item()
        return recon, training_loss
    
def train_landmark_model(ae, landmark_trainloader):
    landmark_train_recon, landmark_train_final_loss = ae.train_landmark_model(args.epochs, landmark_trainloader)
    f = open('landmark_train_final_loss.pckl', 'wb')
    pickle.dump(landmark_train_final_loss, f)
    f.close()
#     f = open('landmark_train_final_loss.pckl', 'rb')
#     landmark_train_final_loss = pickle.load(f)
#     f.close()
    
    print('Final Epoch Loss for Training Landmark Model: ', landmark_train_final_loss)
    return landmark_train_final_loss

def train_appear_model_with_warped_imgs(ae, images_train_warped):
    warped_face_trainset = dataset_constructor(images_train_warped, transform=transforms.Compose([
                                                                    ImgToTensor()]))
    warped_face_trainloader = torch.utils.data.DataLoader(warped_face_trainset, \
                                                    batch_size=args.batch_size, \
                                                    shuffle=True, \
                                                    num_workers=2)
    images_train_recon, images_train_final_loss, ae = ae.train_appear_model(args.epochs, warped_face_trainloader)
    f = open('images_train_final_loss.pckl', 'wb')
    pickle.dump(images_train_final_loss, f)
    f.close()
#     f = open('images_train_final_loss.pckl', 'rb')
#     images_train_final_loss = pickle.load(f)
#     f.close()

    print('images_train_recon: ', np.shape(images_train_recon), torch.max(images_train_recon[0]), torch.min(images_train_recon[0]))
    print('Final Epoch Loss for Training Appearance Model (with warped images): ', images_train_final_loss)
    return images_train_final_loss, ae

def recon_appear_warped_test_imgs(ae, images_test_warped):
    warped_face_testset = dataset_constructor(images_test_warped, transform=transforms.Compose([
                                                                    ImgToTensor()]))
    warped_face_testloader = torch.utils.data.DataLoader(warped_face_testset, \
                                                    batch_size=args.batch_size, \
                                                    shuffle=True, \
                                                    num_workers=2)
    face_testset_recon_appear, face_testset_training_loss = ae.test_appear_model(warped_face_testloader)
    print('Loss for Reconstructing Appearance of Warped Test Images: ', face_testset_training_loss)
    return face_testset_recon_appear

def recon_geometry_test_landmarks(ae, landmark_testloader):
    landmark_test_recon, landmark_test_training_loss = ae.test_landmark_model(landmark_testloader)
    print('Loss for Reconstructing Geometry of Test Landmarks: ', landmark_test_training_loss)
    return landmark_test_recon

def calc_mean(A):
    A_sum = np.zeros(np.shape(A[0]))
    for item in A:
        A_sum += item
    return A_sum / len(A)

def warp_imgs_to_mean(imgs, landmark_train, landmark_mean):
    warped_imgs = []
    for i, img in enumerate(imgs):
        warped_img = warp(img, landmark_train[i], landmark_mean)
        warped_imgs.append(warped_img)
    return warped_imgs

def warp_imgs_back(face_testset_recon_appear, landmark_mean, landmark_test_recon):
    unwarped_imgs = [] 
    for batch_i, batch in enumerate(face_testset_recon_appear):
        for i, img in enumerate(batch):
            img = img.permute(1, 2, 0).detach().numpy()
            recon_landmark = np.reshape(landmark_test_recon[batch_i][i].detach().numpy(), (68, 2))
            unwarped_img = warp(img, landmark_mean, recon_landmark)
            unwarped_imgs.append(unwarped_img)
    return unwarped_imgs

def disp_recon_imgs(recon_imgs, orig_imgs):
    f, axes = plt.subplots(4, 5)
    for i, img in enumerate(recon_imgs):
        img = img.clip(min=0, max=1)
        row = math.floor(i / 5)
        axes[row, i % 5].imshow(img)
    plt.show()
    
    f, axes = plt.subplots(4, 5)
    for i, img in enumerate(orig_imgs):
        row = math.floor(i / 5)
        axes[row, i % 5].imshow(img)
    plt.show()

def reshape_landmarks(landmarks):
    results = []
    for landmark in landmarks:
        flat = np.asarray(landmark).flatten()
        results.append(flat)
    return results

def run_autoencoder():
    face_trainset = dataset_constructor(images_train, transform=transforms.Compose([
                                                                    ImgToTensor()]))
    face_testset = dataset_constructor(images_test, transform=transforms.Compose([
                                                                    ImgToTensor()]))
    face_trainloader = torch.utils.data.DataLoader(face_trainset, \
                                                    batch_size=args.batch_size, \
                                                    shuffle=True, \
                                                    num_workers=2)
    face_testloader = torch.utils.data.DataLoader(face_testset, \
                                                    batch_size=args.batch_size, \
                                                    shuffle=False, \
                                                    num_workers=2)
    
    landmark_train_mod = reshape_landmarks(landmark_train)
    landmark_trainset = dataset_constructor(landmark_train_mod, transform=transforms.Compose([
                                                                    LandmarkToTensor()]))
    landmark_test_mod = reshape_landmarks(landmark_test)
    landmark_testset = dataset_constructor(landmark_test_mod, transform=transforms.Compose([
                                                                    LandmarkToTensor()]))
    landmark_trainloader = torch.utils.data.DataLoader(landmark_trainset, \
                                                        batch_size=args.batch_size, \
                                                        shuffle=True, \
                                                        num_workers=2)
    landmark_testloader = torch.utils.data.DataLoader(landmark_testset, \
                                                        batch_size=args.batch_size, \
                                                        shuffle=False, \
                                                        num_workers=2)

    # ----------------------- Autoencoder: Question 1 ------------------------
    # Train a landmark model with training landmarks
    # ae = autoencoder(args.appear_lr, args.landmark_lr, 0)
    # landmark_train_final_loss = train_landmark_model(ae, landmark_trainloader)
    
    print('here now')
    # Warp training images into the mean position
    landmark_mean = calc_mean(landmark_train)
    images_train_warped = warp_imgs_to_mean(images_train, landmark_train, landmark_mean)

    # Train appearance model with warped images
    # images_train_final_loss, ae = train_appear_model_with_warped_imgs(ae, images_train_warped)
#     f = open('autoencoder.pckl', 'wb')
#     pickle.dump(ae, f)
#     f.close()
    print('loading autoencoder')
    f = open('autoencoder.pckl', 'rb')
    ae = pickle.load(f)
    f.close()
    print('done loading autoencoder')

    # Warp test images into the mean position
    images_test_warped = warp_imgs_to_mean(images_test, landmark_test, landmark_mean)
    
    # Reconstruct appearance of warped test images using appear_model(images_test_warped)
    face_testset_recon_appear = recon_appear_warped_test_imgs(ae, images_test_warped)
    
    # Reconstruct geometry of test landmarks using landmark_model(test_landmarks)
    landmark_test_recon = recon_geometry_test_landmarks(ae, landmark_testloader)
    
    # Warp the warped test images back from the mean position to reconstructed test landmarks
    recon_faces = warp_imgs_back(face_testset_recon_appear, landmark_mean, landmark_test_recon)

    # Plot 20 reconstructed faces and their corresponding original faces
    disp_recon_imgs(recon_faces[:20], images_test[:20])
    
    # Save latent z variables
#     f = open('app_latent_z.pckl', 'wb')
#     pickle.dump(APP_LATENT_Z, f)
#     f.close()
#     f = open('geo_latent_z.pckl', 'wb')
#     pickle.dump(GEO_LATENT_Z, f)
#     f.close()
    f = open('app_latent_z.pckl', 'rb')
    APP_LATENT_Z = pickle.load(f)
    f.close()
    f = open('geo_latent_z.pckl', 'rb')
    GEO_LATENT_Z = pickle.load(f)
    f.close()

    print('Status check: ')
    print('length, shape of APP_LATENT_Z: ', len(APP_LATENT_Z), np.shape(APP_LATENT_Z))
    print('length, shape of GEO_LATENT_Z: ', len(GEO_LATENT_Z), np.shape(GEO_LATENT_Z))
    
    # ----------------------- Autoencoder: Question 2 ------------------------

def main():
    run_autoencoder()

args = parser.parse_args(args=[])
args.cuda = torch.cuda.is_available()
torch.cuda.set_device(-1) # was 0

if not os.path.exists(args.path):
    os.makedirs(args.path)
if not os.path.exists(args.log):
    os.makedirs(args.log)

if args.cuda:
    torch.cuda.manual_seed(args.seed)

face_images_reader = data_reader(args.image_dir, 6, '000000', '.jpg')
images_train, images_test = face_images_reader.read(split=800, \
                                                                read_type='image')

face_landmark_reader = data_reader(args.landmark_dir, 6, '000000', '.mat')
landmark_train, landmark_test = face_landmark_reader.read(split=800, \
                                                                read_type='landmark')

if __name__ == "__main__":
    main()


here now
loading autoencoder
done loading autoencoder
Loss for Reconstructing Appearance of Warped Test Images:  0.006820490350946784
Loss for Reconstructing Geometry of Test Landmarks:  0.0007941091025713831


<Figure size 640x480 with 20 Axes>

<Figure size 640x480 with 20 Axes>

Status check: 
length, shape of APP_LATENT_Z:  2402 (2402,)
length, shape of GEO_LATENT_Z:  2402 (2402,)
